## 数据预处理

In [1]:
import pandas as pd
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
data = pd.read_csv("data1ramp0.4dt.csv")

In [3]:
data.shape

(2016, 13)

In [4]:
data.head

<bound method NDFrame.head of                 dt  PI_feedwaterPump.y     gain.y  PTarget.y   gain1.y  \
0     0.000000e+00          15426218.0  10.500000        1.0  0.000333   
1     1.000000e-10          15426218.0  10.500000        1.0  0.000333   
2     0.000000e+00          15426218.0  10.500000        1.0  0.000333   
3     1.506334e-01          15287873.0  10.500322        1.0  0.000333   
4     0.000000e+00          15287873.0  10.500322        1.0  0.000333   
...            ...                 ...        ...        ...       ...   
2011  0.000000e+00           2344981.2   4.285714        0.4  0.000333   
2012  1.000000e+01           2344981.8   4.285714        0.4  0.000333   
2013  0.000000e+00           2344981.8   4.285714        0.4  0.000333   
2014  1.000000e+01           2344982.2   4.285714        0.4  0.000333   
2015  0.000000e+00           2344982.2   4.285714        0.4  0.000333   

      GW_CWS1_Valve_Ramp_Cold_HT_out3.y  turbineStress.stress.inStress  \
0      

输入：给水PI_feedwaterPump.y，给煤gain.y，功率指令PTarget.y，速率指令（gain1.y），限制指令GW_CWS1_Valve_Ramp_Cold_HT_out3.y，dt
输出：电功率simpleGenerator.summary.P_el，主气压quadruple2.p，再热压力（quadruple1.p），主蒸汽温度，内应力turbineStress.stress.inStress，外turbineStress.stress.outStress

In [5]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.mean = self.data.iloc[:, 0:12].mean()
        self.std = self.data.iloc[:, 0:12].std()
        self.data.iloc[:, 0:12] = self.data.iloc[:, 0:12].astype(float)
        self.data.iloc[:, 0:12] = (self.data.iloc[:, 0:12] - self.mean) / self.std

    def __len__(self):
        return len(self.data) - 1

    def __getitem__(self, idx):
        x = self.data.iloc[idx + 1, 0:6].values
        # tm_values = tm_values.astype(np.float32)
        y = self.data.iloc[6, 12]

        # 检查数据类型并尝试转换
        try:
            tm = torch.tensor(x, dtype=torch.float32, device=device)
            t0 = torch.tensor(y, dtype=torch.float32, device=device)
        except TypeError:
            raise ValueError(f"Error converting data at index {idx}. tm_values: {x}, t0_value: {y}")

        return tm, t0
        

In [6]:
data.iloc[:,0:6]

,dt,PI_feedwaterPump.y,gain.y,PTarget.y,gain1.y,GW_CWS1_Valve_Ramp_Cold_HT_out3.y
0,0.000000e+00,15426218.0,10.500000,1.0,0.000333,0.02
1,1.000000e-10,15426218.0,10.500000,1.0,0.000333,0.02
2,0.000000e+00,15426218.0,10.500000,1.0,0.000333,0.02
3,1.506334e-01,15287873.0,10.500322,1.0,0.000333,0.02
4,0.000000e+00,15287873.0,10.500322,1.0,0.000333,0.02
...,...,...,...,...,...,...
2011,0.000000e+00,2344981.2,4.285714,0.4,0.000333,0.02
2012,1.000000e+01,2344981.8,4.285714,0.4,0.000333,0.02
2013,0.000000e+00,2344981.8,4.285714,0.4,0.000333,0.02
2014,1.000000e+01,2344982.2,4.285714,0.4,0.000333,0.02


In [7]:
import torch
import torch.nn as nn

class GRUModel(nn.Module):
    def __init__(self, input_size=6, hidden_size=64, output_size=7, num_layers=2):
        super(GRUModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # GRU层
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True  # 输入和输出张量的形状为 (batch, seq, feature)
        )
        
        # 输出层
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_size)
        
        # 初始化隐藏状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # 前向传播 GRU
        out, _ = self.gru(x, h0)
        
        # 解码最后一个时间步的隐藏状态
        out = self.fc(out[:, -1, :])
        
        return out

# 创建模型实例
model = GRUModel(input_size=6, hidden_size=64, output_size=7, num_layers=2)

# 打印模型结构
print(model)

GRUModel(
  (gru): GRU(6, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=7, bias=True)
)


In [8]:
dataset = CustomDataset('data1ramp0.4dt.csv')
dataloader = DataLoader(dataset, batch_size=5, shuffle=False, drop_last=True)

# 数据划分
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# 使用slice而不是random_split
train_dataset = torch.utils.data.Subset(dataset, range(0, train_size))
val_dataset = torch.utils.data.Subset(dataset, range(train_size, train_size + val_size))
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=False, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False, drop_last=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_267056\817424074.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       633421800.0
1       633421800.0
2       633421800.0
3       616475300.0
4       616475300.0
           ...     
2011    244788290.0
2012    244788400.0
2013    244788400.0
2014    244788500.0
2015    244788500.0
Name: simpleGenerator.summary.P_el, Length: 2016, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.data.iloc[:, 0:12] = self.data.iloc[:, 0:12].astype(float)


In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam
criterion = nn.MSELoss()

# 学习率动态调整
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)

# Early stopping
n_epochs_stop = 10
epochs_no_improve = 0
min_val_loss = float('inf')

predicted_vals = []
actual_vals = []

for epoch in range(50):  # max epoch
    model.train()
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        tm, t0 = data
        optimizer.zero_grad()
        outputs = model(tm)
        # 反标准化预测的输出
        outputs_inv_std = inverse_standardize(outputs, dataset.mean[0], dataset.std[0])
        # t0_inv_std = inverse_standardize(t0, dataset.mean_t0, dataset.std_t0)

        loss = criterion(outputs_inv_std, t0)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()



    # Validation loss,Maybe it is ok
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            outputs = model(tm)
            # data取的不对，这不是x——》Y的数据集

            # 反标准化预测的输出
            outputs_inv_std = inverse_standardize(outputs, dataset.mean[0], dataset.std[0])
            # t0_inv_std = inverse_standardize(t0, dataset.mean_t0, dataset.std_t0)
            # t0也是标准化后的，应该反标准化
            # 更改了初始化部分，t0未标准化
            loss = criterion(outputs_inv_std, t0)
            val_loss += loss.item()

            # # 预测与实际，方便出图
            # predicted_vals = outputs_inv_std
            # actual_vals = t0_inv_std
            #
        # if epoch == 9:
        #     predicted_vals.extend(outputs_inv_std.cpu().numpy().tolist())
        #     actual_vals.extend(t0.cpu().numpy().tolist())

    scheduler.step(val_loss)
    # 早停条件
    if val_loss < min_val_loss:
        epochs_no_improve = 0
        min_val_loss = val_loss
    else:
        epochs_no_improve += 1
    if epochs_no_improve == n_epochs_stop:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}, Training Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}")

C:\ProgramData\Anaconda3\envs\OMPRL\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor

In [10]:
tm

tensor([[-0.9934,  1.5866,  1.4073,  1.7046,  0.9998,     nan],
        [-0.9934,  1.5866,  1.4073,  1.7046,  0.9998,     nan],
        [-0.9632,  1.5617,  1.4075,  1.7046,  0.9998,     nan],
        [-0.9934,  1.5617,  1.4075,  1.7046,  0.9998,     nan],
        [-0.9591,  1.5334,  1.4076,  1.7046,  0.9998,     nan]])

In [11]:
tm.shape

torch.Size([5, 6])